In [3]:
# auth_github_app.py
import os, time, jwt, requests
from dotenv import load_dotenv; load_dotenv()

APP_ID = os.environ["GITHUB_APP_ID"]                 # integer string, e.g. "123456"
INSTALLATION_ID = os.environ["GITHUB_INSTALLATION_ID"]
with open(os.environ["GITHUB_APP_PRIVATE_KEY_PEM"], "r") as f:
    PRIVATE_KEY = f.read()

def make_app_jwt(app_id: str, pem: str) -> str:
    now = int(time.time())
    payload = {
        "iat": now - 60,        # 60s in the past for skew
        "exp": now + 540,       # 9 minutes ahead (NOT 10)
        "iss": int(app_id)      # your GitHub App ID (numeric)
    }
    return jwt.encode(payload, pem, algorithm="RS256")

def get_installation_token(installation_id: str, app_jwt: str) -> str:
    url = f"https://api.github.com/app/installations/{installation_id}/access_tokens"
    r = requests.post(url, headers={
        "Authorization": f"Bearer {app_jwt}",
        "Accept": "application/vnd.github+json"
    })
    r.raise_for_status()
    return r.json()["token"]          # 1-hour token

if __name__ == "__main__":
    app_jwt = make_app_jwt(APP_ID, PRIVATE_KEY)
    inst_token = get_installation_token(INSTALLATION_ID, app_jwt)
    print("OK: got installation token (1h lifetime)")


OK: got installation token (1h lifetime)


In [1]:
from github_app_tools import resolve_installation_id, installation_token_cached, ensure_branch, put_file, open_pr, create_issue

README = """# kip-retl-uh-x3l
Bootstrap by AI agents.
"""
CI_YML = """name: ci
on:
  pull_request:
    branches: [ "main" ]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with: { python-version: "3.12" }
      - run: pip install ruff pytest
      - run: ruff check .
      - run: pytest -q || true
"""

if __name__ == "__main__":
    inst_id = resolve_installation_id()
    tok = installation_token_cached(inst_id)
    ensure_branch(tok, base_branch="main", new_branch="ai/dev")
    put_file(tok, "README.md", README, message="chore: add README")
    put_file(tok, ".github/workflows/ci.yml", CI_YML, message="ci: add workflow")
    pr = open_pr(tok, title="Bootstrap: README + CI", body="Adds README and CI workflow.")
    issue = create_issue(tok, title="Plan initial backlog", body="Supervisor: break down MVP into issues.")
    print("PR:", pr["html_url"])
    print("Issue:", issue["html_url"])


PR: https://github.com/Uh-X3L/kip-retl-uh-x3l/pull/4
Issue: https://github.com/Uh-X3L/kip-retl-uh-x3l/issues/5


In [1]:
# app_config.py
import logging
import os
from typing import Optional

from azure.ai.projects.aio import AIProjectClient
from helpers.azure_credential_utils import get_azure_credential
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'helpers.azure_credential_utils'

In [ ]:

# Azure AI Agents Main Execution with Authentication Best Practices

# ---------- Azure Authentication with Multiple Methods ----------
def create_azure_client():
    """Create Azure AI Project client with robust authentication"""
    credential = None
    
    # Method 1: Try Interactive Browser Credential
    try:
        print("🌐 Attempting interactive browser authentication...")
        browser_args = {}
        if AZURE_TENANT_ID:
            browser_args["tenant_id"] = AZURE_TENANT_ID
        
        credential = InteractiveBrowserCredential(**browser_args)
        
        # Test credential
        test_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
        print("✅ Interactive browser authentication successful!")
        return test_client, credential
        
    except Exception as e:
        print(f"⚠️ Interactive browser auth failed: {e}")
    
    # Method 2: Try DefaultAzureCredential
    try:
        print("🔄 Attempting DefaultAzureCredential...")
        default_args = {}
        if AZURE_TENANT_ID:
            default_args["interactive_browser_tenant_id"] = AZURE_TENANT_ID
            
        credential = DefaultAzureCredential(**default_args)
        
        # Test credential
        test_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
        print("✅ DefaultAzureCredential authentication successful!")
        return test_client, credential
        
    except Exception as e:
        print(f"❌ DefaultAzureCredential failed: {e}")
        raise Exception("All authentication methods failed")

# Create Azure client
project_client, credential = create_azure_client()

# ---------- Agent Instructions ----------
SUPERVISOR_INSTRUCTIONS = """You are an AI Development Supervisor agent.

Your responsibilities:
- Convert project backlogs into atomic GitHub Issues with clear acceptance criteria
- Assign work evenly across development agents (dev_a, dev_b, dev_c)  
- Ensure all work happens on 'ai/dev' branch with PRs to 'main'
- Monitor CI/CD status and never auto-merge failing builds
- Request human review for all successful builds before merge
- Use Code Interpreter for data analysis and visualizations when needed

Output format: Use tool calls for GitHub operations and clear messages for team communication."""

DEV_INSTRUCTIONS = """You are an AI Developer agent.

Your responsibilities:
- Implement minimal viable changes for assigned GitHub issues
- Use gh_put_file to create/modify files on 'ai/dev' branch
- Open PRs with gh_open_pr that reference the original issue
- Use Code Interpreter for any calculations, data processing, or visualizations
- Ask Supervisor for clarification when requirements are unclear

Code standards: Follow project conventions, write clean code, include comments."""

# ---------- Create and Run AI Agents ----------
with project_client:
    print("🤖 Creating AI agents with enhanced capabilities...")
    
    # Create Supervisor agent with both GitHub and Code Interpreter tools
    supervisor = project_client.agents.create_agent(
        model=MODEL_DEPLOYMENT_NAME,
        name="supervisor",
        instructions=SUPERVISOR_INSTRUCTIONS,
        tools=github_functions.definitions + code_interpreter.definitions,
    )
    print(f"✅ Created Supervisor agent: {supervisor.id}")

    # Create Developer agents
    dev_agents = []
    for dev_name in ["dev_a", "dev_b", "dev_c"]:
        dev_agent = project_client.agents.create_agent(
            model=MODEL_DEPLOYMENT_NAME,
            name=dev_name,
            instructions=DEV_INSTRUCTIONS,
            tools=github_functions.definitions + code_interpreter.definitions,
        )
        dev_agents.append(dev_agent)
        print(f"✅ Created {dev_name} agent: {dev_agent.id}")

    # Project backlog
    BACKLOG = """
    Project: Solaxy CoinGecko Analysis Platform
    
    MVP Tasks:
    1. Add CONTRIBUTING.md with development guidelines
    2. Create src/index.html with hero section and CTA
    3. Add GitHub issue templates for bugs and features
    4. Create API documentation for Solaxy OHLC endpoints
    5. Add data visualization dashboard prototype
    """

    # Create thread and initiate workflow
    thread = project_client.agents.threads.create()
    print(f"📝 Created thread: {thread.id}")

    # Add initial message
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=f"""Project Backlog:
{BACKLOG}

Instructions:
1. Break down the MVP into atomic GitHub issues
2. Assign issues to development agents
3. Create visualization of project timeline if possible
4. Begin implementation on 'ai/dev' branch
"""
    )
    print(f"✅ Created initial message: {message['id']}")

    # Create and process the run with enhanced monitoring
    print("🚀 Starting AI agent workflow...")
    
    run = project_client.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=supervisor.id,
        additional_instructions="Address the user as 'Project Lead'. This is a production system - ensure quality and thorough testing."
    )
    
    print(f"🏁 Workflow completed with status: {run.status}")
    
    # Enhanced error handling
    if run.status == "failed":
        print(f"❌ Run failed: {run.last_error}")
    elif run.status == "succeeded":
        print("✅ Workflow completed successfully!")

    # Fetch and display all messages with file handling
    print("\n📋 Workflow Messages:")
    messages = project_client.agents.messages.list(thread_id=thread.id)
    
    for message in messages:
        print(f"\n👤 {message.role.upper()}: {message.content}")
        
        # Save any generated images (from Code Interpreter)
        for img in message.image_contents:
            file_id = img.image_file.file_id
            timestamp = int(time.time())
            file_name = f"{timestamp}_{file_id}_workflow_image.png"
            
            try:
                project_client.agents.files.save(file_id=file_id, file_name=file_name)
                print(f"💾 Saved visualization: {Path.cwd() / file_name}")
            except Exception as e:
                print(f"⚠️ Could not save image {file_id}: {e}")

    # Cleanup (optional - comment out for debugging)
    # print("\n🧹 Cleaning up agents...")
    # project_client.agents.delete_agent(supervisor.id)
    # for agent in dev_agents:
    #     project_client.agents.delete_agent(agent.id)
    # print("✅ Agents cleaned up")

print("\n🎉 Azure AI Agent workflow completed!")
print("📊 Check your GitHub repository for new issues and PRs")
print("🖼️ Check current directory for any generated visualizations")


ValueError: Invalid tenant ID provided. You can locate your tenant ID by following the instructions here: https://learn.microsoft.com/partner-center/find-ids-and-domain-names